# Feature Extraction & Fine-Tuning
Performing feature extraction and fine-tuning on a pre-trained Convolutional Neural Network (ConvNet) is a common technique in transfer learning. It allows you to leverage the knowledge and learned representations of a pre-trained model on a large dataset and adapt it to a new task or a smaller dataset.

Fine-tuning enables the model to update the weights of the earlier layers to better adapt to the new task. However, it should be done cautiously with a smaller learning rate to avoid catastrophic forgetting or overfitting.

Freeze the weights of the convolutional base to prevent them from being updated during training. This freezing step ensures that the pre-trained weights are not altered, and you only perform feature extraction using the existing representations. This way, the model acts as a fixed feature extractor.

In [ ]:
import os
from google.colab import files
from google.colab import drive

from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

# Step 1: Load the pre-trained VGG16 model without the classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Step 2: Freeze the weights of the convolutional base
base_model.trainable = False

# Step 3: Create a new model and add the pre-trained convolutional base
model = Sequential()
model.add(base_model)

# Step 4: Add new classification layers on top
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # Update to 1 class

# Step 5: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Mount Google Drive
drive.mount('/content/drive')

In [ ]:
import zipfile

# Step 6: Set the path to your dataset ZIP file in Google Drive
zip_path = '/content/drive/MyDrive/dogs-vs-cats.zip'

# Step 7: Extract the dataset ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dogs-vs-cats')

In [ ]:
train_zip_path = '/content/dogs-vs-cats/train.zip'

with zipfile.ZipFile(train_zip_path, 'r') as train_zip_ref:
    train_zip_ref.extractall('/content/dogs-vs-cats/train')

In [ ]:
test_zip_path = '/content/dogs-vs-cats/test1.zip'

with zipfile.ZipFile(test_zip_path, 'r') as test_zip_ref:
    test_zip_ref.extractall('/content/dogs-vs-cats/test1')

In [ ]:
# Step 8: Set the path to the extracted dataset folder
base_dir = '/content/dogs-vs-cats'
train_data_dir = os.path.join(base_dir, 'train')
test_data_dir = os.path.join(base_dir, 'test1')

In [ ]:
# Step 9: Preprocess your data and create data generators
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary') # Update to binary class mode

# Test data generator
test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

In [ ]:
# Step 10: Train the model (feature extraction)
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10)

In [ ]:
# Step 11: Perform fine-tuning (optional)
base_model.trainable = True

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)

In [ ]:
# Step 12: Evaluate and test the model
evaluation = model.evaluate_generator(test_generator, steps=test_generator.samples // test_generator.batch_size)
print("Test Accuracy: {:.2f}%".format(evaluation[1] * 100))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Step 13: Make predictions on the test data
num_samples = test_generator.samples
batch_size = test_generator.batch_size
steps = num_samples // batch_size + 1  # Add +1 to include the remaining samples

predictions = model.predict_generator(test_generator, steps=steps)[:num_samples]  # Truncate predictions to match true labels
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes[:num_samples]

In [ ]:
# Step 14: Visualize the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(test_generator.class_indices))
class_labels = list(test_generator.class_indices.keys())  # Get the class labels
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

In [ ]:
# Step 15: Print classification report
print('Classification Report:')
print(classification_report(true_labels, predicted_labels, target_names=class_labels))

In [ ]:
# Step 16: Display sample images with predicted labels
plt.figure(figsize=(12, 12))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    image = test_generator[i][0][0]  # Get the first image of the batch
    true_label = class_labels[true_labels[i]]  # Get the true label
    predicted_label = class_labels[predicted_labels[i]]  # Get the predicted label
    plt.imshow(image)
    plt.title(f'True: {true_label}\nPredicted: {predicted_label}')
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
class_indices = test_generator.class_indices
print(class_indices)